In [31]:
# Load env in local
from dotenv import load_dotenv
import os
import getpass

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

os.environ["LANGSMITH_TRACING_V2"] = "true"
if not os.getenv("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
    
print("Loaded OPENAI & LANGSMITH variables")

Loaded OPENAI & LANGSMITH variables


In [32]:
# Load vectordb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

persist_directory = '../docs/chroma/'

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print("Loaded existing vector store.")

Loaded existing vector store.


In [33]:
# Load LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [34]:
# Init graph_builder
from langgraph.graph import MessagesState, StateGraph
graph_builder = StateGraph(MessagesState)

In [35]:
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode

# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    query = state["messages"][-1].content
    retrieved_docs = vectordb.similarity_search(query, k=2)
    
    # Format into prompt
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    system_message_content = (
        "Bạn là nhân viên chăm sóc khách hàng của dự án Masteri Grand View và bạn sẽ trả lời các câu hỏi của khách hàng về dự án Masteri Grand View."
        "Hãy dùng đại từ xưng hô gọi khách hàng là Anh/Chị, còn bạn dùng đại từ xưng hô là Em."
        "Hãy trả lời câu hỏi của khách hàng một cách lịch sự và tôn trọng."
        "Chỉ dựa trên thông tin trong tài liệu dự án Masteri Grand View bên dưới, trả lời câu hỏi:"
        "\n\n"
        f"{docs_content}"
        "\n\n"
        "Trong trường hợp khách hàng hỏi những câu hỏi không liên quan đến Công ty và dự án, hãy từ chối trả lời một cách lịch sự."
        "Nếu thông tin không có trong tài liệu, hãy trả lời với khách hàng rằng thông tin này bạn không rõ và nói khách hàng gọi cho bộ phận kinh doanh"
        "Không bao giờ sáng tạo nội dung"
    )
    conversation_messages = [
        message
        for message in state["messages"]
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [36]:
from langgraph.graph import END
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

# Graph builder
graph_builder.add_node(generate)
graph_builder.set_entry_point("generate")
graph_builder.add_edge("generate", END)
# memory = MemorySaver()
# graph = graph_builder.compile(checkpointer=memory)
graph = graph_builder.compile()

In [41]:
# Execute
query = "Lý do anh nên mua căn hộ dự án"
config = {"configurable": {"thread_id": "abc123"}}
messages = [
    {"role": "user", "content": query}
]
response = await graph.ainvoke({"messages": messages},config=config)
print(response["messages"][-1].content)

Dạ, cảm ơn Anh đã quan tâm đến dự án Masteri Grand View. Dưới đây là những lý do khiến Masteri Grand View trở thành lựa chọn hàng đầu:

1. Tọa lạc trong khu đô thị 117.4ha – The Global City – quỹ đất vàng tại P. An Phú.
2. Là siêu phẩm cao tầng đầu tiên tại trung tâm mới.
3. Vị trí kim cương tại giao điểm đa sắc màu.
4. Được phát triển bởi các tên tuổi hàng đầu thế giới.
5. Thời điểm vàng để sở hữu những căn hộ đầu tiên tại The Global City.
6. Thừa hưởng toàn bộ hệ tiện ích đỉnh cao của The Global City.
7. Sở hữu loại hình 4PN cực khan hiếm trên thị trường.

Nếu Anh cần thêm thông tin chi tiết, em rất sẵn lòng hỗ trợ!


In [42]:
"""Retrieve information related to a query."""

retrieved_docs = vectordb.similarity_search(query, k=2)
serialized = "\n\n".join(
    (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
    for doc in retrieved_docs
)

print(retrieved_docs)

[Document(metadata={'page': 61, 'source': '../knowledge_pdf/masterigrandview.pdf'}, page_content='NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW \nTRỞ THÀNH LỰA CHỌN HÀNG ĐẦU\n1.\n2.\n3.\n4.\n5.\n6.\nTỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –\nTHE GLOBAL CITY –\nQUỸ ĐẤT VÀNG TẠI P. AN PHÚ\nLÀ SIÊU PHẨM CAO TẦNG ĐẦU \nTIÊN \nTẠI TRUNG TÂM MỚI\nVỊ TRÍ KIM CƯƠNG TẠI GIAO \nĐIỂM ĐA SẮC MÀU\nĐƯỢC PHÁT TRIỂN BỞI CÁC \nTÊN TUỔI HÀNG ĐẦU THẾ GIỚI\nTHỜI ĐIỂM VÀNG ĐỂ SỞ HỮU \nNHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE \nGLOBAL CITY\nTHỪA HƯỞNG TOÀN BỘ HỆ TIỆN \nÍCH ĐỈNH CAO CỦA \nTHE GLOBAL CITY\n7.\n8.\nSỞ HỮU LOẠI HÌNH 4PN CỰC KHAN \nHIẾM TRÊN THỊ TRƯỜNG'), Document(metadata={'page': 61, 'source': '../knowledge_pdf/masterigrandview.pdf'}, page_content='NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW \nTRỞ THÀNH LỰA CHỌN HÀNG ĐẦU\n1.\n2.\n3.\n4.\n5.\n6.\nTỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –\nTHE GLOBAL CITY –\nQUỸ ĐẤT VÀNG TẠI P. AN PHÚ\nLÀ SIÊU PHẨM CAO TẦNG ĐẦU \nTIÊN \nTẠI TRUNG TÂM MỚI\nVỊ TRÍ KIM CƯƠNG TẠI GIAO \nĐIỂM ĐA SẮC MÀU\nĐƯ